# Generator

## Generator untuk Retrieval Process

In [1]:
import os
from getpass import getpass
from haystack import Pipeline
from haystack.components.builders import PromptBuilder
from haystack_integrations.components.retrievers.mongodb_atlas import MongoDBAtlasEmbeddingRetriever
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack_integrations.document_stores.mongodb_atlas import MongoDBAtlasDocumentStore
from haystack.components.generators import OpenAIGenerator
from haystack.utils import Secret

/Users/mhdfarhanali/Documents/Farhan SmartShopper/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Environment Setup

if "MONGO_CONNECTION_STRING" not in os.environ:
    os.environ["MONGO_CONNECTION_STRING"] = getpass("Enter your MongoDB connection string: ")

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

# Model embedding
embedder_model = "sentence-transformers/all-mpnet-base-v2"

In [3]:
# Connect to MongoDB Atlas

document_store = MongoDBAtlasDocumentStore(
    database_name="depato_store",
    collection_name="products",
    vector_search_index="vector_index", 
    full_text_search_index="search_index"
)

print("Connected to MongoDB Atlas")

Connected to MongoDB Atlas


In [4]:
# Define Prompt Template

TEMPLATE = """
You are a helpful shopping assistant who helps users find products that best match their query.

You will be given a user query and a list of product documents.
Your task is to summarize the user's intent and recommend the best-matching products.

Return your answer in this format:

<summary_of_query>
<index>. <product_name>
Price: <product_price>
Material: <product_material>
Category: <product_category>
Brand: <product_brand>
Recommendation: <product_recommendation>

Be concise and human-friendly. Focus on making the recommendation sound natural.

The query is: {{query}}

The retrieved products are:
{% for product in documents %}
===========================================================
{{loop.index + 1}}. product_name: {{ product.meta.title }}
product_price: {{ product.meta.price }}
product_material: {{ product.meta.material }}
product_category: {{ product.meta.category }}
product_brand: {{ product.meta.brand }}
product_content: {{ product.content }}
{% endfor %}

===========================================================

Answer:
"""

In [5]:
# Build Retrieval + Generation Pipeline

pipeline = Pipeline()
pipeline.add_component("embedder", SentenceTransformersTextEmbedder(model=embedder_model))
pipeline.add_component("retriever", MongoDBAtlasEmbeddingRetriever(document_store=document_store, top_k=5))
pipeline.add_component("prompt_builder", PromptBuilder(template=TEMPLATE, required_variables=["query", "documents"]))
pipeline.add_component("generator", OpenAIGenerator(model="gpt-4.1", api_key=Secret.from_token(os.environ["OPENAI_API_KEY"])))

# Koneksi antar komponen
pipeline.connect("embedder", "retriever")
pipeline.connect("retriever", "prompt_builder.documents")
pipeline.connect("prompt_builder", "generator")

print("Pipeline constructed successfully!")

Pipeline constructed successfully!


In [6]:
# Run Pipeline

query = "I want to find an outerwear that does not make me hot"

response = pipeline.run(
    {
        "embedder": {"text": query},
        "prompt_builder": {"query": query}
    }
)

Batches: 100%|██████████| 1/1 [00:00<00:00,  9.67it/s]


In [7]:
# Display Results

print("\nQuery:", query)
print("===========================================================\n")

reply = response["generator"]["replies"][0].strip()
print(reply)


Query: I want to find an outerwear that does not make me hot

You’re looking for lightweight outerwear that won’t make you feel hot—something breathable and designed to keep you cool.

5. Under Armour Heatgear Alpha Muscle Crop Tee - Women's  
Price: $35.01  
Material: 90% Polyester, 10% Elastane  
Category: Tops  
Brand: Under Armour  
Recommendation: This crop tee is made with super-breathable mesh and features moisture-wicking technology to keep you cool and dry, making it an excellent choice for outerwear that won’t cause overheating.

2. INGEAR Ladies Rash Guard Long Sleeve Shirt Swimwear  
Price: $16.99  
Material: Unknown  
Category: Tops  
Brand: In Gear  
Recommendation: This lightweight rash guard is designed for water activities but can double as outerwear. It’s well-suited for staying cool, especially outdoors or at the beach, though the exact material isn’t specified.

If staying cool is your top priority, the Under Armour Heatgear Alpha Muscle Crop Tee stands out for its